In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Input, Embedding, Dense, Attention
from tensorflow.keras.models import Model
from tensorflow.keras.utils import pad_sequences
from sklearn.model_selection import train_test_split

In [2]:
# Example data
english_sentences = ['hello', 'how are you', 'good morning']
french_sentences = ['bonjour', 'comment ça va', 'bonjour']



In [13]:
def tokenize(sentences):
    tokenizer  = tf.keras.preprocessing.text.Tokenizer()
    tokenizer.fit_on_texts(sentences)
    return tokenizer, tokenizer.texts_to_sequences(sentences)

In [14]:
eng_tokenizer, eng_sequences = tokenize(english_sentences)

In [15]:
eng_tokenizer

In [16]:
eng_sequences

[[1], [2, 3, 4], [5, 6]]

In [17]:
fr_tokenizer, fr_sequences = tokenize(french_sentences)

In [18]:
fr_sequences

[[1], [2, 3, 4], [1]]

In [21]:
max_eng_len = max(len(seq) for seq in eng_sequences)
max_eng_len

3

In [23]:
max_fr_len = max(len(seq) for seq in fr_sequences)
max_fr_len

3

In [33]:
# Paddding
eng_sequences = pad_sequences(eng_sequences, maxlen=max_eng_len, padding='post')
fr_sequences = pad_sequences(fr_sequences, maxlen=max_fr_len, padding='post')

In [34]:
eng_sequences

array([[1, 0, 0],
       [2, 3, 4],
       [5, 6, 0]])

In [35]:
fr_sequences

array([[1, 0, 0],
       [2, 3, 4],
       [1, 0, 0]])

In [37]:
# Vocabulary sizes
eng_vocab_size = len(eng_tokenizer.word_index)+1
fr_vocab_size = len(fr_tokenizer.word_index)+1

In [38]:
eng_tokenizer.word_index

{'hello': 1, 'how': 2, 'are': 3, 'you': 4, 'good': 5, 'morning': 6}

### Build Encoder-Decoder Model with Attention

In [ ]:
# Hyperparameters
embedding_dim = 64
hidden_dim = 128

# Encoder
encoder_inputs = Input(shape=(max_eng_len,))
encoder_embedding = Embedding(input_dim=eng_vocab_size, output_dim=embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(hidden_state, return_sequences=True, return_sate=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]


# Decoder
decoder_inputs = Input(shape=(max_fr_len,))
decoder_embedding = Embedding(input_dim=fr_vocab_size, output_dim=embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(units=hidden_dim, return_sequences=True, return_state=True )
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

# Attention layer
attention=Attention()
context_vector = attention([decoder_outputs, encoder_outputs])

